In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

## 패키지 설치 및 Drive 연결

In [2]:
!pip install PyDrive
# pip install torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torchvision --upgrade --index-url https://download.pytorch.org/whl/cu118
# !pip install --upgrade torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python
!pip install opencv-python-headless
!pip install torchvision
!pip install google-api-core==2.11.0 google-api-python-client==2.79.0 google-auth==2.16.1 google-auth-httplib2==0.1.0 google-auth-oauthlib==0.4.1 googleapis-common-protos==1.58.0
!pip install --upgrade google-auth-oauthlib
!pip install requests google-auth-oauthlib google-auth-httplib2
# !pip install --force-reinstall sympy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.1%2Bcu118-cp310-cp310-linux_x86_64.whl (857.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp310-cp310-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached 

In [3]:
# YOLOv8 모델  설치
!pip3 install tqdm
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
# 라이브러리 불러오기
import os
import json
import shutil
from PIL import Image
import re
import time
import pickle
import tempfile
from ultralytics import YOLO

## 구글 드라이브 관련

In [9]:
# Google Drive API를 사용하여 인증 관리
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# Google OAuth 인증 절차
gauth = GoogleAuth()


# 이미 저장된 자격 증명 파일이 있으면 로드
if os.path.exists("credentials.json"):
    gauth.LoadCredentialsFile("credentials.json")
    if gauth.access_token_expired:  # 토큰이 만료되었으면 갱신
        print("토큰 만료로 갱신")
        gauth.Refresh()
    else:  # 유효한 토큰이 있으면 인증 절차 건너뜀
        print("토큰이 있어 패스")
        gauth.Authorize()
else:
    # 자격 증명 파일이 없으면 새로 인증
    gauth.CommandLineAuth() # 이 과정을 통해 브라우저가 열리고, Google 계정 인증이 이루어짐
    gauth.SaveCredentialsFile("credentials.json")  # 자격 증명 파일 저장



토큰이 있어 패스


In [5]:
# Google Drive 객체 생성
drive = GoogleDrive(gauth)

## Drive에서 파일을 불러와 딕셔너리에 저장

### 하위 폴더까지 탐색하는 코드

In [7]:
# 재귀적으로 폴더 탐색하여 파일 목록 가져오기
def recursive_list_files(drive, folder_id):
    files = []
    print(f"폴더 탐색 중: {folder_id}")  # 폴더 ID 출력
    # 현재 폴더 내 파일 및 폴더 목록 가져오기
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 폴더인 경우 재귀 호출
            files.extend(recursive_list_files(drive, file['id']))
        else:
            files.append(file)
    return files

### Drive API 호출 시 에러가 발생하면 일정 시간 대기 후 재시도

In [16]:
from googleapiclient.errors import HttpError

# Google Drive API 호출 시 에러가 발생하면 일정 시간 대기 후 재시도
def drive_list_files_with_retry(drive, folder_id, retries=10, wait_time=5):
    try:
        return drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false", 'maxResults': 400}).GetList()
    except HttpError as error:
        if retries > 0:
            print(f"HttpError 발생: {error}. {wait_time}초 후 재시도합니다... 남은 재시도 횟수: {retries}")
            time.sleep(wait_time)
            return drive_list_files_with_retry(drive, folder_id, retries-1, wait_time*1.5 )  # 대기 시간 증가
        else:
            print(f"HttpError 발생: {error}. 재시도 횟수 초과.")
            raise error

# 캐시 파일에서 데이터 불러오기
def load_cached_files(cache_file):
    if os.path.exists(cache_file):
        print('캐시 파일에서 데이터 불러오기...')
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

# 캐시 파일 저장하기
def save_cached_files(files, cache_file):
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())  # 현재 시간 포맷
    # 작업 중인 디렉토리에서 임시 파일 생성
    temp_file = None
    try:
        with tempfile.NamedTemporaryFile(delete=False, dir='.') as temp_file:
            pickle.dump(files, temp_file)

        # 임시 파일을 캐시 파일로 이동 (덮어쓰기)
        os.replace(temp_file.name, cache_file)
        print(f"[{current_time}] 캐시 저장 완료. 현재까지 탐색된 파일 수: {len(files)}")

    except Exception as e:
        print(f"캐시 파일 저장 중 오류 발생: {e}")
        if temp_file and os.path.exists(temp_file.name):
            os.remove(temp_file.name)  # 임시 파일 삭제
            

# 재귀적으로 폴더 탐색하여 파일 목록 가져오기
def recursive_list_files(drive, folder_id, cache_file, files=None, save_interval=500):
    if files is None:
        files = []
        # 여기 코드가 None이여도 load_cached_files를 실행하도록 했었음....
        
    print(f"폴더 탐색 중: {folder_id}")  # 폴더 ID 출력
    
    # 현재 폴더 내 파일 및 폴더 목록 가져오기
    file_list = drive_list_files_with_retry(drive, folder_id)
    # file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 폴더인 경우 재귀 호출
            files = recursive_list_files(drive, file['id'],cache_file, files, save_interval)
        else:
            files.append(file)

        # 일정 간격(save_interval)마다 캐시 파일로 저장
        if len(files) % save_interval == 0:
            save_cached_files(files,cache_file)

    return files

### 라벨 데이터 캐싱 + 딕셔너리에 저장하는 코드

In [9]:
# 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


start_time = time.time()

# 라벨 및 원천 데이터의 최상위 폴더 ID
training_folder_id = '1nor9jn1J30mZN_168SsNj1ArO4duu1VT'


# 캐시 파일 경로 설정
label_cache_file = 'sample_lane_label_files.pkl'



# Google Drive에서 가져온 폴더 ID 
label_folder_id = '1ov8fVkcQXS61iboq_lSf3mdHk9ttD6qL'  # 라벨링 데이터 폴더 ID를 수동으로 설정 

# 캐시 파일이 있는지 확인하고 불러오기
label_files = load_cached_files(label_cache_file)

if label_files is None:  # 캐시 파일이 없는 경우 Google Drive에서 데이터 가져오기
    print('캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...')
    if label_folder_id:
        # 라벨 파일 가져오기
        print('라벨쪽 실행 시작...')
        label_files = recursive_list_files(drive, label_folder_id, label_cache_file)
        print(f"라벨 파일 수: {len(label_files)}")

        # 파일 목록을 캐시 파일에 저장
        save_cached_files(label_files, label_cache_file)
    else:
        print("라벨 폴더의 ID를 찾을 수 없습니다.")
        label_files = []
else:
    print(f"캐시 파일에서 불러온 라벨 파일 수: {len(label_files)}")

# 캐시된 데이터에서 딕셔너리 생성
label_dict = {file['title']: file['id'] for file in label_files if file['title'].endswith('.json')}


# 라벨 파일 목록 일부 출력
print("라벨 파일 목록 일부:", list(label_dict.items())[:5])

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...
라벨쪽 실행 시작...
폴더 탐색 중: 1ov8fVkcQXS61iboq_lSf3mdHk9ttD6qL
폴더 탐색 중: 1OTOrtKfiRUq9Kw2ldwwrMnrzLeg_Z21D
폴더 탐색 중: 1cTg5Pbe5h9R_94Vl2KH3N6wNQQjDVrE2
폴더 탐색 중: 1rLnLxLkmUznaQJiBCNjDJZC42FXBynHC
폴더 탐색 중: 1-0cIwiXZfLKSnpA3k6d-VYdI_7xG22sM
폴더 탐색 중: 1-4_gdXYcZR_karf7Z6fS5WTjgUxNTX9H
폴더 탐색 중: 1-ATSbk3X4hSN5DNtK8AyMNNV0XIyTnbz
폴더 탐색 중: 1-FODAIU7kNF1jDMw1DkVNnOuVZaonJ_e
폴더 탐색 중: 1-H6ejxuG1Wc-AEpyUvesuPKwPMkATJ8U
폴더 탐색 중: 1-KipR9XM5ZNuvFXmuj4r2phxq5TgOE3R
폴더 탐색 중: 1-LxApuFk9A9ehoUFzlc4rTuaTnQDCPGi
폴더 탐색 중: 1-NilAKbDooImB0nSs8FBe5d8I3tTXvXV
폴더 탐색 중: 1-TV8KIoXrFu1PqumKoNh2-E0c9-V_n9A
폴더 탐색 중: 1i9kKWW5jDN50YWnb8EfQDmW0xXcyTnen
폴더 탐색 중: 1LXx6MVVkB2DNw38qsww_S2xWDmF38VN-
폴더 탐색 중: 1f7Z_eA2EILxIXOXTbOxmjLBiDVje-1am
폴더 탐색 중: 179JfMQQdUvUxYq6J4l8NADm8YSxb6vOF
폴더 탐색 중: 1QkqQyjbfLCzCBp8Emqsou-INXDn4Q9bP
폴더 탐색 중: 1rxRhaZcn4_wkjfjx-GACMZJR0-1wMVBK
폴더 탐색 중: 1epgIKY983GY2_DBy62WA0gEX6E1akOxY
폴더 탐색 중: 1evsd4fhKuKIuWEBnDRYg4nb4pt43Kdtg
폴더 탐색 중: 1UsJ8HM6S5uIOFGtj6W8pf2NRhs3sViho

In [ ]:
print("라벨 파일 목록 일부:", list(label_dict.items())[60690:])
print(len(label_dict))

### 원천 데이터 캐싱 + 딕셔너리에 저장하는 코드

In [17]:
# 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


start_time = time.time()

# 라벨 및 원천 데이터의 최상위 폴더 ID
training_folder_id = '1nor9jn1J30mZN_168SsNj1ArO4duu1VT'

# 캐시 파일 경로 설정
image_cache_file = 'sample_lane_image_files.pkl'

# Google Drive에서 가져온 폴더 ID
image_folder_id = '1-023uBMup5pgPnePeKi7pnTA1ccKBx1K'  # 원천 데이터 폴더 ID를 수동으로 설정 


# 캐시 파일이 있는지 확인하고 불러오기
image_files = load_cached_files(image_folder_id)

if image_folder_id:
    # 원천 파일 가져오기
    print('원천쪽 실행시작...')
    image_files = recursive_list_files(drive, image_folder_id, image_cache_file)
    print(f"사진 파일 수: {len(image_files)}")

    # 파일 목록을 캐시 파일에 저장
    save_cached_files(image_files, image_cache_file)
else:
    print("이미지 폴더의 ID를 찾을 수 없습니다.")
    image_files = []


# 캐시된 데이터에서 딕셔너리 생성
image_dict = {file['title']: file['id'] for file in image_files if file['title'].endswith('.jpg')}

print("이미지 파일 목록 일부:", list(image_dict.items())[:5])

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


원천쪽 실행시작...
폴더 탐색 중: 1-023uBMup5pgPnePeKi7pnTA1ccKBx1K
폴더 탐색 중: 1EfjaKQfBduhgMQh3mLKfhp6NP7sUf1wS
폴더 탐색 중: 1Mf1KbFVGamEU9RMZJ9_npXH6gavZ_HC-
폴더 탐색 중: 1SrlACEdRzNJ3ijLAASHLYDR4EwjvGAPt
폴더 탐색 중: 1H-IzSLWzlipq099kucY-pnMFWzrl6-rq
폴더 탐색 중: 1HSdbtGQpElc_QhO0oRbWZrtWKjhBX2hR
폴더 탐색 중: 1GDcbX7qMdRCVNuIqt267TAalbUMU_USo
폴더 탐색 중: 1GU4tYwttWsqVde8HYWgA3OQKqythswQu
폴더 탐색 중: 1DvQFSNUhOrHxbiai-W4FVGeYkcOjzQ9i
폴더 탐색 중: 1EgpFs_y31oR9zX4OXfSfIKzwaCkosghC
폴더 탐색 중: 17r3ppGcX5VZHMeAXWylmXHquHsGPb7bm
폴더 탐색 중: 18H83Di6rtBCgS1H4dU8vaLx0TCUyi9BW
폴더 탐색 중: 13UYN52DUxydO5JOQPfrxy2aEWpObLX20
폴더 탐색 중: 13IjY4Ebk7d6jiAMVb7ztHTstPdtmk7Qc
폴더 탐색 중: 1-2yX_8uuDlHFW5DiHXzvUiEyji-0iqPb
폴더 탐색 중: 1zia3D9uU-0oixSvYvIiRZW8eZ1xmnSUw
폴더 탐색 중: 1yLToCUfueLWzHoWBzKMVrv5twLcnSvny
폴더 탐색 중: 1z4MvK1nOlRy6k8JecNU8a4TejSj-Te2f
폴더 탐색 중: 1xng2lOalOi_CZaqV15hozsZ3LgcOQ1w-
폴더 탐색 중: 1y4KRrY2PQNs1FcflooG2o-ngPKax08CV
폴더 탐색 중: 1wbrHT51fYygZQkfBeiMVEIwF6_2ThbwZ
폴더 탐색 중: 1wyGbZerb2dFnc3RC3XbYpctUSEJ9acfX
폴더 탐색 중: 1w4dVlIsQ0fSOj3a1NfZeYd1pEDQjs_MJ

In [19]:

# pkl 파일을 로드하는 함수
def load_pkl_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

# pkl 파일 경로 지정
file_path = 'sample_lane_image_files.pkl'
# pkl 파일의 데이터를 불러와서 출력
data = load_pkl_file(file_path)
print(data[:5])  # 데이터의 타입 확인

[GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'owner', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1NExkoYkbEaZ_6Ogv1safW8DjgLUNn47Z/permissions/me', 'etag': '"k3vzsG1_DmSRvhWWdUdr5lXnVPU"', 'pendingOwner': False}, 'fileExtension': 'jpg', 'md5Checksum': '9c5531266b1aaad434c15cc50adbe6e0', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1NExkoYkbEaZ_6Ogv1safW8DjgLUNn47Z', 'ownerNames': ['승우'], 'lastModifyingUserName': '승우', 'editable': True, 'writersCanShare': True, 'downloadUrl': 'https://www.googleapis.com/drive/v2/files/1NExkoYkbEaZ_6Ogv1safW8DjgLUNn47Z?alt=media&source=downloadUrl', 'mimeType': 'image/jpeg', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1NExkoYkbEaZ_6Ogv1safW8DjgLUNn47Z/parents/1H-IzSLWzlipq099kucY-pnMFWzrl6-rq', 'id': '1H-IzSLWzlipq099kucY-pnMFWzrl6-rq', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https://www.googleapis.com/drive

## 데이터셋 구성하기

In [29]:
# 2. CustomDataset 클래스를 사용하는 방식
# 모델 학습을 위한 데이터로더 생성
import cv2
import json
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from PIL import Image  # OpenCV 이미지 -> PIL 이미지 변환에 필요

class CustomDataset(Dataset):
    def __init__(self, drive, image_dict, label_dict, transform=None):
        self.drive = drive
        self.image_dict = image_dict
        self.label_dict = label_dict
        self.transform = transform
        self.image_names = list(image_dict.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_id = self.image_dict[image_name]

        print(f"처리 중인 이미지 : {image_name}") # 이미지 이름 출력

        # 이미지 파일 읽기
        file = self.drive.CreateFile({'id': image_id})
        file.GetContentFile('temp_image.jpg')  # 임시 파일로 저장
        img = cv2.imread('temp_image.jpg')  # OpenCV로 이미지 읽기

        # OpenCV 이미지를 PIL 이미지로 변환 (transforms 사용을 위해)
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        
        # 이미지 전처리
        if self.transform:
            img = self.transform(img)

        # 라벨 파일 읽기
        label_name = image_name.replace('.jpg', '.json')
        if label_name in self.label_dict:
            label_id = self.label_dict[label_name]
            label_file = self.drive.CreateFile({'id': label_id})
            label_file.GetContentFile('temp_label.json')  # 임시 파일로 저장

            # 파일이 비어 있는지 확인
            if os.path.getsize('temp_label.json') ==0:
                print(f"라벨 파일이 비어 있습니다: {label_name}")
                return img, None


            # JSON 파일 파싱
            try:            
                with open('temp_label.json', 'r', encoding='utf-8') as f:
                    label_json = json.load(f)
            except json.JSONDecodeError:
                print(f"JSONDecodeError: 라벨 파일이 유효하지 않습니다: {label_name}")
                return img, None

            # JSON에서 필요한 데이터 추출
            image_path = label_json.get('imagePath')
            value = label_json.get('value')
            label_id = label_json.get('id')

            # target에 필요한 정보 저장
            target = {'imagePath': image_path, 'value': value, 'id': label_id}
            # 바운딩 박스와 클래스 ID를 텐서로 변환
            # 필요한 형식에 맞게 변환하세요
            # 예를 들어, bbox 좌표를 정규화하거나 텐서로 변환
            # ...
        else:
            print(f"라벨 파일을 찾을 수 없습니다: {label_name}")
            target = None

        return img, target



## 데이터셋 생성 및 모델 학습

In [22]:
from ultralytics import YOLO  # YOLOv8 모델 가져오기
# YOLOv8 모델 로드
model = YOLO('yolov8n.pt') 

In [23]:
import time
start_time = time.time()

# YOLOv8 모델을 훈련 모드로 전환
model.train()

# 전처리 정의 (리사이즈 후 텐서로 변환)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 모든 이미지를 224x224 크기로 리사이즈
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

# 데이터셋 및 데이터로더 생성
dataset = CustomDataset(drive, image_dict, label_dict, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0) # num_workers=0 : Drive와의 데이터 통신 문제 줄이기 위함 -> 단일 프로세스로 데이터 로딩 수행
num_epochs = 1

# 모델 학습 루프
for epoch in range(num_epochs):
    for imgs, targets in dataloader:
        # imgs와 targets를 모델에 입력
        # imgs: 이미지 텐서
        # targets: 바운딩 박스와 클래스 ID 등

        # 모델에 맞게 데이터 형식 조정
        # ...

        # 모델 출력 및 손실 계산
        outputs = model(imgs, targets)
        loss = outputs['loss'] # 모델이 예측한 결과로부터 손실 계산
        # loss = loss_function(outputs, targets)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")

New https://pypi.org/project/ultralytics/8.2.101 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 169M/169M [00:02<00:00, 62.0MB/s] 
Unzipping /home/j-j11a303/datasets/coco2017labels-segments.zip to /home/j-j11a303/datasets/coco...: 100%|██████████| 122232/122232 [00:14<00:00, 8568.95file/s]


Dataset download success ✅ (2380.8s), saved to /home/j-j11a303/datasets



100%|██████████| 755k/755k [00:00<00:00, 61.2MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

...

                   all       5000      36335       0.62      0.466       0.51      0.361

100 epochs completed in 22.608 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:35<00:00,  4.41it/s]


                   all       5000      36335       0.62      0.466       0.51      0.361
                person       2693      10777      0.761      0.663       0.74       0.51
               bicycle        149        314      0.665      0.376      0.428      0.258
                   car        535       1918      0.647      0.507      0.553      0.352
            motorcycle        159        367      0.717      0.569       0.65      0.412
              airplane         97        143      0.776      0.755      0.802      0.614
                   bus        189        283      0.781       0.65      0.742      0.616
                 train        157        190      0.775      0.762      0.829      0.636
                 truck        250        414      0.557      0.376      0.432      0.289
                  boat        121        424      0.579      0.307      0.387      0.211
         traffic light        191        634      0.654      0.343      0.399      0.204
          fire hydran

Defaulting to user installation because normal site-packages is not writeable

requirements: AutoUpdate success ✅ 2.1s, installed 1 package: ['pycocotools>=2.0.6']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
Loading and preparing results...
DONE (t=3.09s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=70.85s).
Accumulating evaluation results...
DONE (t=17.81s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.364
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.515
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.398
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | max

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [26]:
# 학습된 모델 로드
model = YOLO('runs/detect/train/weights/best.pt')

# 검증 수행
model.val(data='coco.yaml')

Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /home/j-j11a303/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:49<00:00,  6.32it/s]


                   all       5000      36335      0.625      0.463       0.51      0.361
                person       2693      10777      0.767      0.659       0.74       0.51
               bicycle        149        314      0.675      0.379       0.43       0.26
                   car        535       1918      0.657      0.504      0.552      0.353
            motorcycle        159        367      0.722      0.578      0.658      0.413
              airplane         97        143        0.8      0.754      0.806      0.618
                   bus        189        283      0.784      0.647      0.742      0.614
                 train        157        190      0.771      0.747      0.828      0.636
                 truck        250        414      0.554      0.375      0.432      0.289
                  boat        121        424      0.599      0.304      0.383      0.211
         traffic light        191        634      0.652      0.339      0.395      0.204
          fire hydran

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efed3306650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024

## pkl을 통해 CustomDataset 만들기

In [56]:
import requests

# pkl 파일을 로드하는 함수
def load_pkl_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# 이미지 다운로드 함수 (Google Drive API에서 이미지 다운로드)
# def download_file(download_url, save_path):
#     response = requests.get(download_url)
#     if response.status_code == 200:
#         with open(save_path, 'wb') as f:
#             f.write(response.content)
#     else:
#         print(f"파일 다운로드 실패: {download_url}")
#     return save_path
# 메모리 상에서 처리
def download_image(download_url):
    response = requests.get(download_url)
    if response.status_code == 200:
        img_array = np.frombuffer(response.content, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        return img
    else:
        print(f"파일 다운로드 실패: {download_url}")
        return None

# # 라벨 다운로드 함수
# def download_label(download_url):
#     response = requests.get(download_url)
#     if response.status_code == 200:
#         try:
#             label_json = json.loads(response.content.decode('utf-8'))
#             return label_json
#         except json.JSONDecodeError:
#             print(f"라벨 파일 유효하지 않음: {download_url}")
#             return None
#     else:
#         print(f"라벨 다운로드 실패: {download_url}")
#         return None

# YOLO 형식의 바운딩 박스 계산 함수 (폴리곤을 감싸는 최소 직사각형)
# YOLO 모델은 바운딩 박스 좌표를 이미지의 상대 좌표로 사용
def polygon_to_bbox(points, img_width, img_height):
    x_coords = [point['x'] for point in points]
    y_coords = [point['y'] for point in points]
    
    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)
    
    # YOLO 형식으로 변환 (x_center, y_center, width, height)
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return x_center, y_center, width, height

# JSON 파일에서 YOLO 형식의 라벨 정보 추출
def extract_label_info(json_data):
    labels = []

    for obj in json_data['data_set_info']['data']:
        metainfo = obj['value'].get('metainfo', {})
        points = obj['value'].get('points', [])
        object_label = obj['value'].get('object_Label', '')        
        # 클래스 매핑
        class_mapping = {
            'vehicle_car': 0,
            'lane_blue': 1,
            'license_plate': 2,
            'no_parking_sign': 3
        }
        class_id = class_mapping.get(object_label, None)
        if class_id is not None and points:
            bbox = polygon_to_bbox(points, img_width, img_height)
            labels.append([class_id, *bbox])
    
        # metainfo = obj['value']['metainfo']
        # points = obj['value']['points']
        # object_label = obj['value']['object_Label']
        
        # # 차량일 경우 YOLO 라벨 형식 (차량 유형을 class 번호로 변환)
        # if 'vehicle_type' in object_label:
        #     class_id = 0  # 예: vehicle_car는 0번 class
        #     bbox = polygon_to_bbox(points)
        #     labels.append([class_id, *bbox])

        # # 차선일 경우 YOLO 라벨 형식 (차선 유형을 class 번호로 변환)
        # elif 'lane_type' in object_label:
        #     class_id = 1  # 예: lane_blue는 1번 class
        #     bbox = polygon_to_bbox(points)
        #     labels.append([class_id, *bbox])
        
        # # 번호판일 경우 YOLO 라벨 형식 (번호판을 위한 class 번호)
        # elif 'value' in obj and obj['value'] == 'license_plate':
        #     class_id = 2  # 번호판 class
        #     bbox = polygon_to_bbox(points)
        #     labels.append([class_id, *bbox])
        
        # # 불법 주정차 표지판일 경우
        # elif 'value' in obj and obj['value'] == 'no_parking_sign':
        #     class_id = 3  # 불법 주정차 표지판 class
        #     bbox = polygon_to_bbox(points)
        #     labels.append([class_id, *bbox])
    
    return labels


# # YOLO 라벨을 텍스트 파일로 저장하는 함수
# def save_yolo_labels(labels, output_path):
#     with open(output_path, 'w') as f:
#         for label in labels:
#             class_id, x_center, y_center, width, height = label
#             f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# # JSON 파일을 로드하고 라벨 추출 및 저장
# def process_json_to_yolo(json_file_path, output_label_path):
#     with open(json_file_path, 'r', encoding='utf-8') as f:
#         json_data = json.load(f)

#     labels = extract_label_info(json_data)
#     save_yolo_labels(labels, output_label_path)


# CustomDataset 클래스
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list, transform=None):
        self.image_list = image_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = self.image_list[idx]  # 이미지 정보
        label = self.label_list[idx]


        # 이미지 다운로드 및 로드
        # img = download_image(image)
        # if img is None:
        #     return None, None

        # img_height, img_width = img.shape[:2]
        
        # OpenCV 이미지를 PIL 이미지로 변환 (transforms 적용을 위해)
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        # 이미지 전처리
        if self.transform:
            img = self.transform(img)


        # 라벨 다운로드 및 파싱
        if label_info:
            label_json = download_label(label_info['downloadUrl'])
            if label_json is not None:
                target = extract_label_info(label_json, img_width, img_height)
                target = torch.tensor(target, dtype=torch.float32)
            else:
                target = torch.zeros((0, 5), dtype=torch.float32)
        else:
            print(f"라벨 파일 없음: {image_name}")
            target = torch.zeros((0, 5), dtype=torch.float32)

        return img, target



In [57]:
start_time = time.time()

# 이미지 전처리 설정 (리사이즈 후 텐서로 변환)
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLOv8에 맞게 이미지 크기 조정
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

# pkl 파일에서 이미지와 라벨 데이터 로드
image_list = load_pkl_file('sample_lane_image_files.pkl')
label_list = load_pkl_file('sample_lane_label_files.pkl')


# CustomDataset과 DataLoader 생성
dataset = CustomDataset(image_list, label_list, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)

# YOLOv8 모델에 데이터 로드
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # YOLOv8 모델 로드

# 옵티마이저와 손실 함수 설정
optimizer = torch.optim.Adam(model.model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
epochs = 2
    
# 모델 학습 루프
for epoch in range(epochs):  # 에포크 수 설정
    model.model.train()
    for imgs, targets in dataloader:
        # None 값 제거
        imgs = [img for img in imgs if img is not None]
        targets = [target for target in targets if target is not None]

        if len(imgs) == 0:
            continue

        imgs = torch.stack(imgs).to(model.device)
        # targets는 리스트이므로 필요한 형태로 변환해야 합니다.

        optimizer.zero_grad()
        preds = model.model(imgs)

        # 손실 계산 (여기서는 간단히 설명하므로 실제 구현에서는 수정이 필요합니다)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} 완료")

# 모델 저장
model.model.save('custom_yolov8n.pt')

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


# # 모델을 학습하는 대신 데이터 로더에서 배치를 받아 처리
# for batch in dataloader:
#     imgs, labels = batch
#     results = model(imgs)  # YOLO 모델에 이미지 입력

#     # 모델 학습 코드 (custom loss나 training loop을 직접 구성 가능)
#     model.train_on_batch(imgs, labels)  # 모델 학습 함수



# 모델 학습
# model.train(data='custom_dataset.yaml', epochs=100)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [34]:
        # # 라벨 파일 다운로드 및 파싱
        # if label_info:
        #     label_json = download_label(label_info['downloadUrl'])
        #     try:
        #         with open(label_save_path, 'r', encoding='utf-8') as f:
        #             label_json = json.load(f)
        #     except json.JSONDecodeError:
        #         print(f"라벨 파일 유효하지 않음: {label_save_path}")
        #         return img, None


        #     target = extract_label_info(label_json)
        # else:
        #     print(f"라벨 파일 없음: {image_name}")
        #     target = None

<class 'list'>
<class 'list'>


## 로컬 드라이브에서 데이터 연결

In [5]:
# 1. 이미지와 라벨 파일 경로 수집 함수
def get_all_files(root_dir, exts):
    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(exts):
                file_path = os.path.join(root, file)
                # 경로에서 'TL', 'TS1', 'TS2', 'TS3' 이후의 경로를 추출
                relative_path = extract_relative_path(file_path)
                if relative_path:
                    file_list.append((relative_path, file_path))
    return file_list
    
# 2. 상대 경로 추출 함수 ('TL', 'TS1', 'TS2', 'TS3' 이후의 경로)
def extract_relative_path(file_path):
    match = re.search(r'(TL|TS1|TS2|TS3)[\\/](.+)', file_path)
    if match:
        return match.group(2)  # 'TL', 'TS1', 'TS2', 'TS3' 이후의 경로 반환
    else:
        return None

# 3. 이미지와 라벨 디렉토리 경로
# 드라이브에서 폰으로 변경!!!!!
# image_root_dir = r'G:\내 드라이브\Colab Notebooks\SpecializedProject\content\차로 위반 영상 데이터\sample_datas\원천데이터'
# label_root_dir = r'G:\내 드라이브\Colab Notebooks\SpecializedProject\content\차로 위반 영상 데이터\sample_datas\라벨링데이터'

# 4. 이미지와 라벨 파일 리스트 생성
image_files = get_all_files(image_root_dir, ('.jpg', '.png'))
label_files = get_all_files(label_root_dir, ('.json',))

# 5. 라벨 파일 딕셔너리 생성 (상대 경로를 키로)
label_dict = {relative_path: file_path for relative_path, file_path in label_files}

# 6. 이미지와 라벨 파일 매칭
matched_files = []
for relative_path, image_file in image_files:
    if relative_path in label_dict:
        label_file = label_dict[relative_path]
        matched_files.append((image_file, label_file))
    else:
        print(f"라벨 파일이 없습니다: {relative_path}")
        
# 7. 클래스 목록 정의
classes = ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike',
           'lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder',
           'no_parking_sign']

# 8. YOLO 형식 라벨 저장 디렉토리 생성
yolo_labels_dir = 'dataset/labels/train'
os.makedirs(yolo_labels_dir, exist_ok=True)

# 9. 이미지 저장 디렉토리 생성
yolo_images_dir = 'dataset/images/train'
os.makedirs(yolo_images_dir, exist_ok=True)

# 10. JSON 라벨을 YOLO 형식으로 변환 및 저장
def convert_json_to_yolo(json_path, img_width, img_height, classes):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    yolo_labels = []
    for obj in data['data_set_info']['data']:
        value = obj['value']
        object_label = value['object_Label']
        
        # 클래스 이름 추출
        if 'vehicle_type' in object_label:
            class_name = object_label['vehicle_type']
        elif 'lane_type' in object_label:
            class_name = object_label['lane_type']
        elif 'no_parking_sign' in object_label:
            class_name = 'no_parking_sign'
        else:
            # 필요한 경우 다른 클래스 처리 추가
            continue  # 클래스 정보가 없으면 스킵
        
        if class_name not in classes:
            continue  # 클래스 목록에 없으면 스킵
        class_id = classes.index(class_name)
        
        # 바운딩 박스 좌표 추출
        points = value['points']
        xs = [point['x'] for point in points]
        ys = [point['y'] for point in points]
        x_min = min(xs)
        x_max = max(xs)
        y_min = min(ys)
        y_max = max(ys)
        
        # YOLO 형식으로 변환
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_labels

# 11. 매칭된 이미지와 라벨 파일을 처리하여 YOLO 데이터셋 생성
for image_file, label_file in matched_files:
    # 이미지 열기
    image = Image.open(image_file)
    img_width, img_height = image.size

    # JSON 라벨을 YOLO 형식으로 변환
    yolo_labels = convert_json_to_yolo(label_file, img_width, img_height)

    # 이미지 파일 저장
    image_name = os.path.basename(image_file)
    dest_image_path = os.path.join(yolo_images_dir, image_name)
    shutil.copy(image_file, dest_image_path)

    # 라벨 파일 저장
    label_name = os.path.splitext(image_name)[0] + '.txt'
    dest_label_path = os.path.join(yolo_labels_dir, label_name)
    with open(dest_label_path, 'w') as f:
        f.write('\n'.join(yolo_labels))

# # 12. 데이터셋 설정 파일(custom_dataset.yaml) 작성
# yaml_content = f"""
# train: dataset/images/train
# val: dataset/images/val  # 검증 데이터가 없으면 train 데이터로 대체

# nc: {len(classes)}
# names: {classes}
# """

# with open('custom_dataset.yaml', 'w') as f:
#     f.write(yaml_content)


# # 2. CustomDataset 클래스 구현
# class CustomDataset(Dataset):
#     def __init__(self, images_dir, labels_dir, classes,  transform=None):
#         self.images_dir = images_dir
#         self.labels_dir = labels_dir
#         self.transform = transform
#         self.classes = classes
#         self.image_names = [f for f in os.listdir(images_dir)
#                             if f.endswith('.jpg')]

#     def __len__(self):
#         return len(self.image_names)

#     def __getitem__(self, idx):
#         image_name = self.image_names[idx]
#         image_path = os.path.join(self.images_dir, image_name)


#         # 이미지 로드
#         image = Image.open(image_path).convert('RGB')
#         img_width, img_height = image.size
        
#         # 이미지 전처리
#         if self.transform:
#             image = self.transform(image)

#         # 라벨 파일 로드 및 YOLO 형식으로 변환
#         label_name = image_name.replace('.jpg', '.json').replace('.png', '.json')
#         label_path = os.path.join(self.labels_dir, label_name)
#         if not os.path.exists(label_path):
#             print(f"라벨 파일이 없습니다: {label_path}")
#             labels = torch.zeros((0, 5))  # 빈 라벨
#         else:
#             yolo_labels = convert_json_to_yolo(label_path, img_width,
#                                                img_height, self.classes)
#             labels = []
#             for yolo_label in yolo_labels:
#                 label = [float(x) for x in yolo_label.strip().split()]
#                 labels.append(label)
#             labels = torch.tensor(labels)

#         return img, target

NameError: name 'image_root_dir' is not defined

## 원격 server 에서 이미지 바로 처리


In [5]:
# 1. 이미지와 라벨 파일 경로 수집 함수
def get_all_files(root_dir, exts):
    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(exts):
                file_path = os.path.join(root, file)
                # 경로에서 root_dir을 제거하고 상대 경로를 추출
                relative_path = os.path.relpath(file_path, root_dir)  # root_dir을 기준으로 상대 경로 추출
                file_list.append((relative_path, file_path))
    return file_list
    
# 3. 이미지와 라벨 디렉토리 경로
image_root_dir = 'dataset/lane_violation/sample_datas/Image_datas/TL'
label_root_dir = 'dataset/lane_violation/sample_datas/Labeling_datas/TL'

# 4. 이미지와 라벨 파일 리스트 생성
image_files = get_all_files(image_root_dir, ('.jpg', '.png'))
label_files = get_all_files(label_root_dir, ('.json',))

# 5. 라벨 파일 딕셔너리 생성 (상대 경로를 키로, 확장자 제외)
label_dict = {os.path.splitext(relative_path)[0]: file_path for relative_path, file_path in label_files}

# 6. 이미지와 라벨 파일 매칭
matched_files = []
# 파일 확장자 제외하고 비교
for relative_path, image_file in image_files:
    image_name_without_ext = os.path.splitext(relative_path)[0]  # 확장자 제거
    if image_name_without_ext in label_dict:
        label_file = label_dict[image_name_without_ext]
        matched_files.append((image_file, label_file))
    else:
        print(f"라벨 파일이 없습니다: {relative_path}")
        print(f"이미지 파일 경로: {image_file}")
        print(f"라벨 파일이 존재하는 경로들: {list(label_dict.keys())[:5]}")  # 일부 라벨 파일 경로 출력
        
# # 7. 클래스 목록 정의
# classes = ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike',
#            'lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder']

# 7. 클래스 목록 정의 (8개의 클래스)
# vehicle_attribute에 따른 확장
classes = [
    # 'vehicle_car_normal', 'vehicle_car_danger', 'vehicle_car_violation',
    # 'vehicle_bus_normal', 'vehicle_bus_danger', 'vehicle_bus_violation',
    'vehicle_car','vehicle_bus','vehicle_truck', 'vehicle_bike',
    'lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder'
]

# 8. YOLO 형식 라벨 저장 디렉토리 생성
yolo_labels_dir = 'dataset/labels/train'
os.makedirs(yolo_labels_dir, exist_ok=True)

# 9. 이미지 저장 디렉토리 생성
yolo_images_dir = 'dataset/images/train'
os.makedirs(yolo_images_dir, exist_ok=True)

# 10. JSON 라벨을 YOLO 형식으로 변환 및 저장
def convert_json_to_yolo(json_path, img_width, img_height, classes):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    yolo_labels = []
    for obj in data['data_set_info']['data']:
        value = obj['value']
        object_label = value['object_Label']
        # # 클래스 이름과 속성(vehicle_attribute)을 추출
        # if 'vehicle_type' in object_label:
        #     class_name = object_label['vehicle_type']

        
        #     # vehicle_attribute에 따라 클래스를 구분
        #     if 'vehicle_attribute' in object_label:
        #         attribute = object_label['vehicle_attribute']
        #         if class_name == 'vehicle_car':
        #             if attribute == 'normal':
        #                 class_name = 'vehicle_car_normal'
        #             elif attribute == 'danger':
        #                 class_name = 'vehicle_car_danger'
        #             elif attribute == 'violation':
        #                 class_name = 'vehicle_car_violation'
        #         elif class_name == 'vehicle_bus':
        #             if attribute == 'normal':
        #                 class_name = 'vehicle_bus_normal'
        #             elif attribute == 'danger':
        #                 class_name = 'vehicle_bus_danger'
        #             elif attribute == 'violation':
        #                 class_name = 'vehicle_bus_violation'
        #         # truck과 bike는 attribute 없이 사용
        #         elif class_name == 'vehicle_truck':
        #             class_name = 'vehicle_truck'
        #         elif class_name == 'vehicle_bike':
        #             class_name = 'vehicle_bike'
                    
        # elif 'lane_type' in object_label:
        #     class_name = object_label['lane_type']

        # vehicle_type과 lane_type만 처리
        if 'vehicle_type' in object_label:
            class_name = object_label['vehicle_type']
        elif 'lane_type' in object_label:
            class_name = object_label['lane_type']
        else:
            continue  # 클래스 정보가 없으면 스킵
        
        if class_name not in classes:
            continue  # 클래스 목록에 없으면 스킵
        class_id = classes.index(class_name)
        
        # # 차량 클래스와 vehicle_attribute에 따른 클래스 설정
        # if 'vehicle_type' in object_label:
        #     vehicle_type = object_label['vehicle_type']
        #     vehicle_attribute = object_label['vehicle_attribute']  # 정상, 위험, 위반

        #     # vehicle_attribute에 따른 클래스 이름 변경
        #     if vehicle_type == 'vehicle_car':
        #         class_name = f"{vehicle_type}_{vehicle_attribute}"  # 예: vehicle_car_normal
        #     elif vehicle_type == 'vehicle_bus':
        #         class_name = f"{vehicle_type}_{vehicle_attribute}"  # 예: vehicle_bus_violation
        #     else:
        #         class_name = vehicle_type  # truck이나 bike는 attribute가 없으므로 기본 처리
        # # 차선 정보
        # elif 'lane_type' in object_label:
        #     class_name = object_label['lane_type']

        # else:
        #     continue # 클래스 정보가 없으면 스킵
            
        # # 클래스가 정의된 목록에 있을 때만 처리
        # if class_name not in classes:
        #     continue
        # class_id = classes.index(class_name)
        
        # 바운딩 박스 좌표 추출
        points = value['points']
        xs = [point['x'] for point in points]
        ys = [point['y'] for point in points]
        x_min = min(xs)
        x_max = max(xs)
        y_min = min(ys)
        y_max = max(ys)
        
        # YOLO 형식으로 변환
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_labels

# 11. 매칭된 이미지와 라벨 파일을 처리하여 YOLO 데이터셋 생성
for image_file, label_file in matched_files:
    # 이미지 열기
    image = Image.open(image_file)
    img_width, img_height = image.size

    # JSON 라벨을 YOLO 형식으로 변환
    yolo_labels = convert_json_to_yolo(label_file, img_width, img_height, classes)

    # 이미지 파일 저장
    image_name = os.path.basename(image_file)
    dest_image_path = os.path.join(yolo_images_dir, image_name)
    shutil.copy(image_file, dest_image_path)

    # 라벨 파일 저장
    label_name = os.path.splitext(image_name)[0] + '.txt'
    dest_label_path = os.path.join(yolo_labels_dir, label_name)
    with open(dest_label_path, 'w') as f:
        f.write('\n'.join(yolo_labels))


In [7]:
start_time = time.time()


# 13. 모델 학습
model = YOLO('yolov8n.pt')
model.train(data='custom_dataset.yaml', epochs=10, imgsz=640)

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")

New https://pypi.org/project/ultralytics/8.3.3 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=custom_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train36, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, 

train: Scanning /home/j-j11a303/dataset/labels/train.cache... 1363 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1363/1363 [00:00<?, ?it/s]
val: Scanning /home/j-j11a303/dataset/labels/train.cache... 1363 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1363/1363 [00:00<?, ?it/s]


Plotting labels to runs/detect/train36/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train36
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      2.24G     0.8315      2.514      1.005          6        640: 100%|██████████| 86/86 [00:11<00:00,  7.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:07<00:00,  5.44it/s]


                   all       1363       2733      0.967      0.159      0.321       0.25

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      2.26G     0.4757     0.5557      0.845          6        640: 100%|██████████| 86/86 [00:09<00:00,  9.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:07<00:00,  6.13it/s]

                   all       1363       2733      0.954      0.584      0.588      0.525



10 epochs completed in 0.049 hours.
Optimizer stripped from runs/detect/train36/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train36/weights/best.pt, 6.2MB

Validating runs/detect/train36/weights/best.pt...
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:08<00:00,  5.12it/s]


                   all       1363       2733      0.954      0.584      0.588      0.525
           vehicle_car        798        803      0.954      0.988      0.991      0.904
           vehicle_bus        518        520      0.961      0.975      0.988      0.901
         vehicle_truck         34         34      0.859      0.717      0.735      0.633
          vehicle_bike         13         13      0.912          1      0.995      0.847
            lane_white         14         14          1          0    0.00963    0.00888
             lane_blue       1343       1343      0.947      0.994      0.982      0.905
           lane_yellow          2          2          1          0          0          0
         lane_shoulder          4          4          1          0          0          0
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train36
코드 실행 시간: 315.05초


In [34]:
import os

train_path = "G:/내 드라이브/Colab Notebooks/SpecializedProject/content/차로 위반 영상 데이터/134.차로 위반 영상 데이터/01.data/1.Training/원천데이터"
print(os.path.exists(train_path))

False


In [70]:
import os
print(os.name)

posix


In [8]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

start_time = time.time()

# 1. 학습된 best.pt 모델 로드
model = YOLO('best.pt')

# 2. 예측하고자 하는 이미지 파일 경로
image_path = '[BLUE]00039A_171232_005.jpg'

# 3. 모델에 이미지 입력 후 예측 수행
results = model.predict(source=image_path, save=True, imgsz=640, conf=0.05)

# 4. 결과 출력
print("결과창이야!!!!",results)
print("결과창 끝이야!!!!")

# 5. 결과 이미지 저장 및 출력
results[0].plot()  # 예측 결과를 이미지에 시각적으로 그립니다.
# 저장된 이미지를 불러오기 위해 경로를 직접 지정해야 할 수 있습니다.
# result_image_path = 'runs/detect/predict7/' + image_path.split('/')[-1]  # 저장된 이미지 경로 추정


# 6. 저장된 이미지 불러와서 출력
result_image = cv2.imread(image_path)
# plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
plt.imshow(result_image)
plt.axis('off')
plt.show()

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


image 1/1 /home/j-j11a303/[BLUE]00127A_101213_004.jpg: 384x640 1 vehicle_car, 1 vehicle_bus, 2 lane_blues, 34.9ms
Speed: 3.3ms preprocess, 34.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict31
결과창이야!!!! [ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'vehicle_car', 1: 'vehicle_bus', 2: 'vehicle_truck', 3: 'vehicle_bike', 4: 'lane_white', 5: 'lane_blue', 6: 'lane_yellow', 7: 'lane_shoulder'}
obb: None
orig_img: array([[[ 10,   5,   2],
        [  5,   0,   0],
        [ 17,  11,   6],
        ...,
        [119, 115, 110],
        [119, 115, 110],
        [119, 115, 110]],

       [[ 12,   7,   4],
        [ 22,  17,  14],
        [ 69,  63,  58],
        ...,
        [119, 115, 110],
        [119, 115, 110],
        [119, 115, 110]],

       [[  5,   0,   0],
        [ 10,   5,   2],
        [ 61,  55,  50],
        ...,
        [11

<Figure size 640x480 with 1 Axes>

코드 실행 시간: 0.37초


In [101]:
# 저장된 이미지 경로를 results 객체에서 확인
print(results[0].boxes)  # 예측된 바운딩 박스 정보
# print(results[0].save_dir)  # 이미지가 저장된 디렉토리 경로


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))


In [113]:
# 1. 학습된 best.pt 모델 로드
model = YOLO('best.pt')

# 2. 예측하고자 하는 이미지 파일 경로
image_path = '[BLUE]00352A_155210_005.jpg'

# 3. 모델에 이미지 입력 후 예측 수행
results = model.predict(source=image_path, save=True, imgsz=640, conf=0.01)

# 4. 이미지가 저장된 경로 확인
print(f"이미지 저장 경로: {results[0].save_dir}")

# 5. 결과 이미지 저장 및 출력
results[0].plot()  # 예측된 바운딩 박스를 이미지에 표시

# 6. 저장된 이미지 경로 설정
result_image_path = f"{results[0].save_dir}/{image_path}"  # 이미지가 저장된 경로로 수정

# 7. 저장된 이미지 불러와서 출력 (BGR에서 RGB로 변환)
result_image = cv2.imread(result_image_path)
if result_image is None:
    print("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
else:
    # 이미지 크기 설정 (옵션)
    plt.figure(figsize=(10, 10))
    
    # 이미지 색상 변환 후 출력
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.show()



image 1/1 /home/j-j11a303/[BLUE]00352A_155210_005.jpg: 384x640 1 lane_blue, 9.1ms
Speed: 2.5ms preprocess, 9.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict30
이미지 저장 경로: runs/detect/predict30


<Figure size 1000x1000 with 1 Axes>

In [114]:
# 4. 원본 이미지 불러오기
image = cv2.imread(image_path)

# 5. 탐지된 바운딩 박스 정보 추출
for box in results[0].boxes:
    # 바운딩 박스 좌표 추출 (xyxy 형식)
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
    
    # 클래스 이름 및 신뢰도 추출
    class_id = int(box.cls[0])
    confidence = box.conf[0]
    
    # 클래스 이름 추출
    class_name = results[0].names[class_id]
    
    # 바운딩 박스 그리기 (OpenCV 사용)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 녹색 사각형
    cv2.putText(image, f'{class_name} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

<Figure size 1000x1000 with 1 Axes>

In [117]:
image_path_test = os.path.abspath('[BLUE]00352A_155210_005.jpg')
# 절대 경로 확인
print(f"이미지 절대 경로: {image_path_test}")

# 3. 모델에 이미지 입력 후 예측 수행
results = model.predict(source=image_path_test, save=True, imgsz=640, conf=0.01)

# 4. 원본 이미지 불러오기
image = cv2.imread(image_path_test)

if image is None:
    print("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
else:
    # 이미지 출력 (BGR -> RGB 변환)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

이미지 절대 경로: /home/j-j11a303/[BLUE]00352A_155210_005.jpg

image 1/1 /home/j-j11a303/[BLUE]00352A_155210_005.jpg: 384x640 1 lane_blue, 7.1ms
Speed: 2.7ms preprocess, 7.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict30


<Figure size 640x480 with 1 Axes>

In [118]:
# 7. 저장된 이미지 불러와서 출력
result_image_test = cv2.imread(image_path_test)
print(results[0].boxes) 

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([9.], device='cuda:0')
conf: tensor([0.3898], device='cuda:0')
data: tensor([[1.0043e+03, 6.0373e+02, 1.5396e+03, 1.0800e+03, 3.8979e-01, 9.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1271.9298,  841.8631,  535.3307,  476.2738]], device='cuda:0')
xywhn: tensor([[0.6625, 0.7795, 0.2788, 0.4410]], device='cuda:0')
xyxy: tensor([[1004.2645,  603.7262, 1539.5952, 1080.0000]], device='cuda:0')
xyxyn: tensor([[0.5231, 0.5590, 0.8019, 1.0000]], device='cuda:0')


In [24]:
# 1. 학습된 best.pt 모델 로드
model = YOLO('runs/detect/train36/weights/best.pt')

# 2. 예측하고자 하는 이미지 파일 경로
image_path = '[BLUE]00127A_101213_004.jpg'

# 이미지 예측
results = model.predict(source=image_path , save=True, imgsz=640)  # your_image.jpg 경로에 맞게 수정

# 결과 확인
for result in results:
    result.plot()  # 예측 결과 이미지 출력


image 1/1 /home/j-j11a303/[BLUE]00127A_101213_004.jpg: 384x640 1 vehicle_car, 1 vehicle_bus, 1 lane_blue, 12.3ms
Speed: 3.3ms preprocess, 12.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict40


In [ ]:
print(results)

In [25]:
# 4. 차량과 차선 객체 분류
vehicle_boxes = []
lane_boxes = []

for box in results[0].boxes:  # results[0]을 사용하여 첫 번째 결과에 접근
    cls = int(box.cls)  # 클래스 인덱스 가져오기
    class_name = results[0].names[cls]  # 클래스 이름 가져오기
    
    # 클래스 이름에 따라 차량과 차선을 구분
    if class_name in ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike']:
        vehicle_boxes.append(box)
    elif class_name in ['lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder']:
        lane_boxes.append(box)

# 5. 차량과 차선 간의 관계 분석
def is_overlapping(box1, box2):
    # 각 경계 상자의 좌표 (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1.xyxy[0].tolist()  # 차량 경계 상자 좌표
    x1_2, y1_2, x2_2, y2_2 = box2.xyxy[0].tolist()  # 차선 경계 상자 좌표
    
    # 겹침 여부 확인 (두 상자가 겹치는 경우 True 반환)
    return not (x2_1 < x1_2 or x2_2 < x1_1 or y2_1 < y1_2 or y2_2 < y1_1)


print((vehicle_boxes))

# 6. 불법 주차 여부 판단
for vehicle_box in vehicle_boxes:
    vehicle_coords = vehicle_box.xyxy  # 차량의 경계 상자 좌표
    
    for lane_box in lane_boxes:
        lane_coords = lane_box.xyxy  # 차선의 경계 상자 좌표
        
        # 차량이 차선 위에 있는지 확인 (경계 상자 간의 겹침 여부 계산)
        if is_overlapping(vehicle_box, lane_box):
            lane_type = results[0].names[int(lane_box.cls)]  # 차선의 종류
            print(f"차량이 {lane_type} 차선 위에 있습니다.")
            
            # 파란색 차선 위에 있으면 불법 주차로 판단
            if lane_type == 'lane_blue':
                print("불법 주차입니다.")
            else:
                print("정상 주차입니다.")

[ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.], device='cuda:0')
conf: tensor([0.7470], device='cuda:0')
data: tensor([[1.1333e+03, 6.2856e+02, 1.7388e+03, 1.0211e+03, 7.4698e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1436.0541,  824.8519,  605.5305,  392.5768]], device='cuda:0')
xywhn: tensor([[0.7479, 0.7638, 0.3154, 0.3635]], device='cuda:0')
xyxy: tensor([[1133.2888,  628.5635, 1738.8193, 1021.1404]], device='cuda:0')
xyxyn: tensor([[0.5903, 0.5820, 0.9056, 0.9455]], device='cuda:0'), ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.5308], device='cuda:0')
data: tensor([[1.1311e+03, 6.2092e+02, 1.7453e+03, 1.0185e+03, 5.3079e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1438.1819,  819.7084,  614.1616,  397.5734]], device='cuda:0')
